# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

You will need a new person a specific version of OPENAI to use with the dashboard in pannel


In [ ]:
!pip install openai
!pip install panel
!pip install python-dotenv
!pip install jupyter_bokeh

  Using cached jiter-0.6.0-cp312-none-win_amd64.whl.metadata (5.3 kB)
   ---------------------------------------- 0.0/383.7 kB ? eta -:--:--
   -- ------------------------------------ 20.5/383.7 kB 682.7 kB/s eta 0:00:01
   --------- ------------------------------ 92.2/383.7 kB 1.1 MB/s eta 0:00:01
   ----------------------- ---------------- 225.3/383.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 383.7/383.7 kB 2.4 MB/s eta 0:00:00
Using cached jiter-0.6.0-cp312-none-win_amd64.whl (199 kB)
  Using cached jupyter_bokeh-4.0.5-py3-none-any.whl.metadata (7.1 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached jupyter_bokeh-4.0.5-py3-none-any.whl (148 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [22]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [24]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [26]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course!


In [28]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [30]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I'm a text-based chatbot and I don't have the ability to remember personal information about users. How can I assist you today?


In [31]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

The code below is the updated version of the dashboard.

In [1]:
import panel as pn  # GUI
pn.extension()

import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

client = openai.ChatCompletion 

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    
    response = client.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response['choices'][0]['message']['content']

panels = []
context = [
    {'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You greet the customer, collect the order, and ask if it's a pickup or delivery. \
Once you have the entire order, you summarize it and ask if the customer \
wants to add anything else. For delivery, you ask for an address. \
Finally, you collect payment. Be friendly and conversational. \
The menu includes:
Pepperoni pizza: $12.95, $10.00, $7.00
Cheese pizza: $10.95, $9.25, $6.50
Eggplant pizza: $11.95, $9.75, $6.75
Fries: $4.50, $3.50
Greek salad: $7.25
Toppings:
Extra cheese: $2.00, Mushrooms: $1.50, Sausage: $3.00, 
Canadian bacon: $3.50, AI sauce: $1.50, Peppers: $1.00
Drinks:
Coke: $3.00, $2.00, $1.00, Sprite: $3.00, $2.00, $1.00, Bottled water: $5.00
"""}
]

def collect_messages(event):
    prompt = inp.value
    inp.value = ''  
    context.append({'role': 'user', 'content': prompt})
    
    response = get_completion(prompt)
    context.append({'role': 'assistant', 'content': response})
    
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    
    chat_panel.objects = panels

inp = pn.widgets.TextInput(value=context[0]['content'], placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

chat_panel = pn.Column(*panels)

button_conversation.on_click(collect_messages)

interactive_panel = pn.Column(
    inp, button_conversation, chat_panel
)

interactive_panel.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'835d3735-2a37-4292-ba78-9c49590fa39a': {'version…

In [2]:
context[0]

{'role': 'system',
 'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You greet the customer, collect the order, and ask if it's a pickup or delivery. Once you have the entire order, you summarize it and ask if the customer wants to add anything else. For delivery, you ask for an address. Finally, you collect payment. Be friendly and conversational. The menu includes:\nPepperoni pizza: $12.95, $10.00, $7.00\nCheese pizza: $10.95, $9.25, $6.50\nEggplant pizza: $11.95, $9.75, $6.75\nFries: $4.50, $3.50\nGreek salad: $7.25\nToppings:\nExtra cheese: $2.00, Mushrooms: $1.50, Sausage: $3.00, \nCanadian bacon: $3.50, AI sauce: $1.50, Peppers: $1.00\nDrinks:\nCoke: $3.00, $2.00, $1.00, Sprite: $3.00, $2.00, $1.00, Bottled water: $5.00\n"}

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
import panel as pn  # GUI
pn.extension()

import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

client = openai.ChatCompletion  

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    
    response = client.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response['choices'][0]['message']['content']

panels = []
context = [
    {'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You greet the customer, collect the order, and ask if it's a pickup or delivery. \
Once you have the entire order, you summarize it and ask if the customer \
wants to add anything else. For delivery, you ask for an address. \
Finally, you collect payment. Be friendly and conversational. \
The menu includes:
Pepperoni pizza: $12.95, $10.00, $7.00
Cheese pizza: $10.95, $9.25, $6.50
Eggplant pizza: $11.95, $9.75, $6.75
Fries: $4.50, $3.50
Greek salad: $7.25
Toppings:
Extra cheese: $2.00, Mushrooms: $1.50, Sausage: $3.00, 
Canadian bacon: $3.50, AI sauce: $1.50, Peppers: $1.00
Drinks:
Coke: $3.00, $2.00, $1.00, Sprite: $3.00, $2.00, $1.00, Bottled water: $5.00
"""}
]

def collect_messages(event):
    prompt = inp.value
    inp.value = ''  
    context.append({'role': 'user', 'content': prompt})
    
    response = get_completion(prompt)
    context.append({'role': 'assistant', 'content': response})
    
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    
    chat_panel.objects = panels

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

chat_panel = pn.Column(*panels)

# Biggest change -> no longer binding, but responding it on click
button_conversation.on_click(collect_messages)

interactive_panel = pn.Column(
    inp, button_conversation, chat_panel
)

interactive_panel.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'d50d9a42-03e5-432c-8914-1c6444da29f1': {'version…

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# Experimentation and Findings

Its fun and it can be used for very intresting applications.

# Report on Variations and Learnings

The variation, it can change the behaivor of the chat and how it responds, asumes roles and how helpfull it can be.